In [47]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [18]:
#Load Data on environmental factors(indicators) per county
df = pd.read_csv("indicators_per-industry_per-county.csv")
df

,Unnamed: 0,fips,county,name,industry_code,industry_detail,relevant_naics,year,payann,estab,...,METL,MINE,MSW,NREN,OZON,PEST,REN,SMOG,VADD,WATR
0,0,8001,1,"Adams County, Colorado",113000,timber and raw forest products,[113],2012,0,1,...,0.000000e+00,0.0,0.000000,0.0,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000
1,1,8001,1,"Adams County, Colorado",115000,agriculture and forestry support,[115],2012,290,6,...,7.965438e-05,0.0,3.405586,0.0,1.827157e-04,8.769207e-03,0.958770,2.011564,211.423463,15.653502
2,2,8001,1,"Adams County, Colorado",211000,unrefined oil and gas,[211],2012,0,6,...,0.000000e+00,0.0,0.000000,0.0,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000
3,3,8001,1,"Adams County, Colorado",2122a0,"iron, gold, silver, and other metal ores","[21221, 21222, 21229]",2012,0,1,...,0.000000e+00,0.0,0.000000,0.0,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000
4,4,8001,1,"Adams County, Colorado",212310,dimensional stone,[21231],2012,0,4,...,0.000000e+00,0.0,0.000000,0.0,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306027,306027,32510,510,"Carson City, Nevada",812300,dry-cleaning and laundry,[8123],2012,0,4,...,0.000000e+00,0.0,0.000000,0.0,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000
306028,306028,32510,510,"Carson City, Nevada",812900,"pet care, photofinishing, parking and other su...",[8129],2012,331,4,...,1.483303e-05,0.0,2.148818,0.0,1.318396e-07,2.876990e-09,0.025598,0.076440,191.834961,0.096584
306029,306029,32510,510,"Carson City, Nevada",813100,religious organizations,[8131],2012,2441,13,...,1.670618e-07,0.0,6.541864,0.0,0.000000e+00,0.000000e+00,0.000000,0.312826,696.409510,2.551873
306030,306030,32510,510,"Carson City, Nevada",813a00,"grantmaking, giving, and social advocacy organ...","[8132, 8133]",2012,635,9,...,8.127071e-09,0.0,4.713797,0.0,0.000000e+00,0.000000e+00,0.000000,0.020206,440.264876,0.051385


In [32]:
#sum over each factor for a county
df_county_indicators=df.groupby(['fips','county','name','year']).sum()
df_county_indicators

,,,,Unnamed: 0,payann,estab,emp,ACID,ENRG,ETOX,EUTR,FOOD,GCC,...,METL,MINE,MSW,NREN,OZON,PEST,REN,SMOG,VADD,WATR
fips,county,name,year,,,,,,,,,,,,,,,,,,,,,
1001,1,"Autauga County, Alabama",2012,22002344,128668,594,4954,299.152569,7.286445e+05,1963.026340,26.933851,709.634497,106259.392004,...,0.050200,0.000000e+00,2149.369209,4.351692e+05,0.002923,3.497395e-02,293475.377416,2949.632736,79914.159702,7851.498268
1003,3,"Baldwin County, Alabama",2012,41609466,953498,3340,32629,133.420049,6.931874e+03,3765.662479,132.635472,4843.498718,43881.445254,...,0.057225,1.674254e+04,16148.087370,4.932628e+03,0.007422,8.577729e-02,1999.246044,4438.579174,576096.081165,1633.902304
1005,5,"Barbour County, Alabama",2012,19847485,53138,361,1715,102.817249,6.064801e+00,3298.083216,13.598555,126.810329,19913.180328,...,0.002351,0.000000e+00,554.606997,0.000000e+00,0.000153,4.371549e-02,6.064801,3649.913530,29611.275038,511.844967
1007,7,"Bibb County, Alabama",2012,16221839,20778,205,655,30.042694,2.012683e+04,1807.410043,4.819015,76.420984,5751.086704,...,0.006445,0.000000e+00,309.053357,0.000000e+00,0.000593,2.437340e-02,20126.828138,1064.780004,10666.884789,285.555832
1009,9,"Blount County, Alabama",2012,23008810,61132,455,2161,32.192658,1.131970e+01,416.998063,4.320160,232.490017,6564.143301,...,0.004599,0.000000e+00,896.650774,0.000000e+00,0.002463,6.501047e-03,11.319704,1165.911030,36511.029249,98.727235
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56037,37,"Sweetwater County, Wyoming",2012,6352509,491651,989,9029,587.935749,3.629382e+06,813.473206,105.117742,1001.624899,205214.901809,...,0.063982,1.394053e+06,4115.339612,3.628234e+06,0.003711,8.917167e-03,1147.804947,19410.599796,312217.341832,5945.474594
56039,39,"Teton County, Wyoming",2012,7237066,361056,1402,8689,37.178584,9.249760e+02,7592.166446,25.091516,2044.202199,14614.797183,...,0.020598,8.200278e+00,6423.591808,4.857347e+02,0.003347,2.019774e-01,439.241290,1412.961358,210925.667302,1777.330068
56041,41,"Uinta County, Wyoming",2012,5811849,145379,442,3287,96.737078,1.919977e+06,112.140420,13.936063,384.841414,32123.362015,...,0.011720,3.997271e+05,1420.508279,1.919961e+06,0.000773,4.621011e-04,16.608615,3486.862556,97322.166731,582.400056


### Clustering for counties based on the environmental factors(indicators)

In [45]:
df_county_indicators.columns
df_factors=df_county_indicators[['ACID', 'ENRG', 'ETOX', 'EUTR',
       'FOOD', 'GCC', 'HAPS', 'HAZW', 'HC', 'HNC', 'HRSP', 'HTOX', 'JOBS',
       'LAND', 'METL', 'MINE', 'MSW', 'NREN', 'OZON', 'PEST', 'REN', 'SMOG',
       'VADD', 'WATR']]
df_factors

,,,,ACID,ENRG,ETOX,EUTR,FOOD,GCC,HAPS,HAZW,HC,HNC,...,METL,MINE,MSW,NREN,OZON,PEST,REN,SMOG,VADD,WATR
fips,county,name,year,,,,,,,,,,,,,,,,,,,,,
1001,1,"Autauga County, Alabama",2012,299.152569,7.286445e+05,1963.026340,26.933851,709.634497,106259.392004,10.732168,15.515155,0.000006,0.000024,...,0.050200,0.000000e+00,2149.369209,4.351692e+05,0.002923,3.497395e-02,293475.377416,2949.632736,79914.159702,7851.498268
1003,3,"Baldwin County, Alabama",2012,133.420049,6.931874e+03,3765.662479,132.635472,4843.498718,43881.445254,31.163349,614.863426,0.000025,0.000110,...,0.057225,1.674254e+04,16148.087370,4.932628e+03,0.007422,8.577729e-02,1999.246044,4438.579174,576096.081165,1633.902304
1005,5,"Barbour County, Alabama",2012,102.817249,6.064801e+00,3298.083216,13.598555,126.810329,19913.180328,7.238123,3.329358,0.000016,0.000062,...,0.002351,0.000000e+00,554.606997,0.000000e+00,0.000153,4.371549e-02,6.064801,3649.913530,29611.275038,511.844967
1007,7,"Bibb County, Alabama",2012,30.042694,2.012683e+04,1807.410043,4.819015,76.420984,5751.086704,2.622439,4.793377,0.000005,0.000030,...,0.006445,0.000000e+00,309.053357,0.000000e+00,0.000593,2.437340e-02,20126.828138,1064.780004,10666.884789,285.555832
1009,9,"Blount County, Alabama",2012,32.192658,1.131970e+01,416.998063,4.320160,232.490017,6564.143301,3.770756,9.737300,0.000005,0.000021,...,0.004599,0.000000e+00,896.650774,0.000000e+00,0.002463,6.501047e-03,11.319704,1165.911030,36511.029249,98.727235
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56037,37,"Sweetwater County, Wyoming",2012,587.935749,3.629382e+06,813.473206,105.117742,1001.624899,205214.901809,46.136744,40.714622,0.000113,0.000664,...,0.063982,1.394053e+06,4115.339612,3.628234e+06,0.003711,8.917167e-03,1147.804947,19410.599796,312217.341832,5945.474594
56039,39,"Teton County, Wyoming",2012,37.178584,9.249760e+02,7592.166446,25.091516,2044.202199,14614.797183,39.699930,167.219233,0.000015,0.000061,...,0.020598,8.200278e+00,6423.591808,4.857347e+02,0.003347,2.019774e-01,439.241290,1412.961358,210925.667302,1777.330068
56041,41,"Uinta County, Wyoming",2012,96.737078,1.919977e+06,112.140420,13.936063,384.841414,32123.362015,10.644368,8.324102,0.000019,0.000117,...,0.011720,3.997271e+05,1420.508279,1.919961e+06,0.000773,4.621011e-04,16.608615,3486.862556,97322.166731,582.400056


In [262]:
#Finding a good setting for eps is sometimes easier after scaling the data
#as using these scaling techniques will ensure that all features have similar ranges.
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_factors)
#calculating appropriate min_points
min_points=len(df_factors.columns)
# cluster the data
dbscan = DBSCAN(eps=0.0001, min_samples = 3)
clusters = dbscan.fit_predict(X_scaled)
# plot the cluster assignments
#plt.scatter(df_factors[:, 0], df_factors[:, 1], c=clusters, cmap="plasma")
#plt.xlabel("Feature 0")
#plt.ylabel("Feature 1")
print(np.unique(clusters))
#clusters.labels_

[-1  0  1  2]


### Clustering for counties based on the HTOX and HC factors

In [78]:
df_county_indicators.columns
df_HTOX_HC=df_county_indicators[['HC', 'HTOX']]
df_HTOX_HC

,,,,HC,HTOX
fips,county,name,year,,
1001,1,"Autauga County, Alabama",2012,0.000006,0.000030
1003,3,"Baldwin County, Alabama",2012,0.000025,0.000135
1005,5,"Barbour County, Alabama",2012,0.000016,0.000078
1007,7,"Bibb County, Alabama",2012,0.000005,0.000035
1009,9,"Blount County, Alabama",2012,0.000005,0.000026
...,...,...,...,...,...
56037,37,"Sweetwater County, Wyoming",2012,0.000113,0.000778
56039,39,"Teton County, Wyoming",2012,0.000015,0.000077
56041,41,"Uinta County, Wyoming",2012,0.000019,0.000136


In [257]:
#Finding a good setting for eps is sometimes easier after scaling the data
#as using these scaling techniques will ensure that all features have similar ranges.
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_HTOX_HC)
#calculating appropriate min_points
min_points=len(df_HTOX_HC.columns)+1
# cluster the data
dbscan = DBSCAN(eps=0.0000008, min_samples = min_points)
clusters = dbscan.fit(X_scaled)
# plot the cluster assignments
#plt.scatter(df_factors[:, 0], df_factors[:, 1], c=clusters, cmap="plasma")
#plt.xlabel("Feature 0")
#plt.ylabel("Feature 1")
#print(np.unique(clusters))
print(np.unique(clusters.labels_))
#clusters.core_sample_indices_
#clusters.components_

[-1  0  1  2]


##### Clustering for counties based on Cancer rates

In [265]:
cancer_df = pd.read_csv("incidencerates.csv")
cancer_df = cancer_df[['locale', 'fips', 'cancer']]  # 'trend_last_5' missing data in 10 counties
df_cancer=cancer_df.groupby(['fips']).sum()
df_cancer

,cancer
fips,
0.0,172748
1000.0,3455
1001.0,3455
1003.0,3455
1005.0,3455
...,...
56037.0,3455
56039.0,3455
56041.0,3455


In [254]:
# cluster the data
dbscan = DBSCAN(eps=1, min_samples = 2)
clusters = dbscan.fit_predict(df_cancer)
# plot the cluster assignments
#plt.scatter(df_factors[:, 0], df_factors[:, 1], c=clusters, cmap="plasma")
#plt.xlabel("Feature 0")
#plt.ylabel("Feature 1")
print(np.unique(clusters))
#clusters.labels_

[-1  0  1  2]


### Random Forest Regressor for environmental factors in a county and cancer rates